# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [1]:
#importing packages and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings 

In [2]:
# Import the data
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [3]:
# Total number of people who survived/didn't survive
df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [4]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = df[relevant_columns]

dummy_dataframe.shape

(891, 7)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [5]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(712, 7)

In [7]:
#getting dummies
dummy_dataframe = pd.get_dummies(dummy_dataframe, columns=['Sex', 'Embarked'], drop_first=True, dtype=float)

In [8]:
dummy_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      712 non-null    int64  
 1   Age         712 non-null    float64
 2   SibSp       712 non-null    int64  
 3   Fare        712 non-null    float64
 4   Survived    712 non-null    int64  
 5   Sex_male    712 non-null    float64
 6   Embarked_Q  712 non-null    float64
 7   Embarked_S  712 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 50.1 KB


Finally, assign the independent variables to `X` and the target variable to `y`: 

In [9]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop('Survived', axis=1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [12]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm
X = sm.add_constant(X)
logit_model = sm.Logit(y,X)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.444229
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [13]:
# Summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      704
Method:                           MLE   Df Model:                            7
Date:                Sun, 31 Aug 2025   Pseudo R-squ.:                  0.3417
Time:                        10:35:41   Log-Likelihood:                -316.29
converged:                       True   LL-Null:                       -480.45
Covariance Type:            nonrobust   LLR p-value:                 5.360e-67
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6378      0.633      8.901      0.000       4.396       6.879
Pclass        -1.2102      0.163     -7.427      0.000      -1.530      -0.891
Age           -0.0433      0.008     -5.263      0.000      -0.059      -0.027
SibSp         -0.3796      0.125     -3.043      0.002      -0.624      -0.135
Fare           0.0012      0.002      0.474      0.635      -0.004       0.006
Sex_male      -2.6168      0.217    -12.040      0.000      -3.043      -2.191
Embarked_Q    -0.8155      0.598     -1.363      0.173      -1.988       0.357
Embarked_S    -0.4036      0.270     -1.494      0.135      -0.933       0.126
==============================================================================
"""

### Your comments here
Most features are statistically significant with p_values < 0.05 in exception of Fare, embarked items

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [18]:
# Your code here
relevant_columns_2 = ['Pclass', 'Age', 'SibSp', 'Sex' , 'Survived']
new_dummy = df[relevant_columns_2]
new_dummy.shape

(891, 5)

In [19]:
#removing null
new_dummy = new_dummy.dropna()
new_dummy.shape

(714, 5)

In [21]:
#getting dummy variables
new_dummy = pd.get_dummies(new_dummy, columns=['Sex'], drop_first=True, dtype=float)

In [22]:
#creating X and y
X = new_dummy.drop('Survived', axis=1)
y = new_dummy['Survived']

In [23]:
#creating model
X = sm.add_constant(X)
lg = sm.Logit(y,X)
results = lg.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.445882
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Sun, 31 Aug 2025   Pseudo R-squ.:                  0.3399
Time:                        10:43:22   Log-Likelihood:                -318.36
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.089e-69
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6008      0.543     10.306      0.000       4.536       6.666
Pclass        -1.3174      0.141     -9.350      0.000      -1.594      -1.041
Age           -0.0444      0.008     -5.442      0.000      -0.060      -0.028
SibSp         -0.3761      0.121     -3.106      0.002      -0.613      -0.139
Sex_male      -2.6235      0.215    -12.229      0.000      -3.044      -2.203
==============================================================================
"""

In [ ]:
# Your comments here
There is no improvement in our R squared.

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!